In [1]:
import sys    # 시스템
import os     # 시스템

import pandas as pd    # 판다스 : 데이터분석 라이브러리
import numpy as np     # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup     # html 데이터 전처리
from selenium import webdriver    # 웹 브라우저 자동화

import time                       # 시간 지연
from tqdm import tqdm_notebook    # 진행상황 표시

In [37]:
# 검색어
keyword1 = input("1.크롤링할 키워드를 입력하세요: ")

'''
#검색할 키워드 정리
10대 여자 어린이날 선물
10대 남자 어린이날 선물

20,30대 엄마 어버이날 선물
20,30대 아빠 어버이날 선물
40,50대 엄마 어버이날 선물
40,50대 아빠 어버이날 선물
60대 엄마 어버이날 선물
60대 아빠 어버이날 선물

20,30대 여자 선생님 스승의날 선물
20,30대  남자 선생님 스승의날 선물
40,50대 여자 선생님 스승의날 선물
40,50대 남자 선생님 스승의날 선물
60대 여자 선생님 스승의날 선물
60대 남자 선생님 스승의날 선물

10대 여자 생일 선물
10대 남자 생일 선물
20,30대 여자 생일 선물
20,30대 남자 생일 선물
40,50대 여자 생일 선물
40,50대 남자 생일 선물
60대 여자 생일 선물
60대 남자 생일 선물

20,30대 여자 결혼기념일 선물
20,30대 남자 결혼기념일 선물
40,50대 여자 결혼기념일 선물
40,50대 남자 결혼기념일 선물
60대 여자 결혼기념일 선물
60대 남자 결혼기념일 선물
'''

1.크롤링할 키워드를 입력하세요: 60대 남자 결혼기념일 선물


'\n#검색할 키워드 정리\n10대 여자 어린이날 선물\n10대 남자 어린이날 선물\n\n20,30대 엄마 어버이날 선물\n20,30대 아빠 어버이날 선물\n40,50대 엄마 어버이날 선물\n40,50대 아빠 어버이날 선물\n60대 엄마 어버이날 선물\n60대 아빠 어버이날 선물\n\n20,30대 여자 선생님 스승의날 선물\n20,30대  남자 선생님 스승의날 선물\n40,50대 여자 선생님 스승의날 선물\n40,50대 남자 선생님 스승의날 선물\n60대 여자 선생님 스승의날 선물\n60대 남자 선생님 스승의날 선물\n\n10대 여자 생일 선물\n10대 남자 생일 선물\n20,30대 여자 생일 선물\n20,30대 남자 생일 선물\n40,50대 여자 생일 선물\n40,50대 남자 생일 선물\n60대 여자 생일 선물\n60대 남자 생일 선물\n\n20,30대 여자 결혼기념일 선물\n20,30대 남자 결혼기념일 선물\n40,50대 여자 결혼기념일 선물\n40,50대 남자 결혼기념일 선물\n60대 여자 결혼기념일 선물\n60대 남자 결혼기념일 선물\n'

수집할 url 크롤링

In [38]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")

# 사이트 주소
driver.get("http://www.naver.com")
time.sleep(1)

# 검색창에 '검색어' 검색
element = driver.find_element_by_id("query") #f12눌러서 확인
element.send_keys(keyword1)
element.submit()
time.sleep(1)

# 'VIEW' 클릭
driver.find_element_by_link_text("VIEW").click()

# '블로그' 클릭
driver.find_element_by_link_text("블로그").click()
time.sleep(1)

# 스크롤 다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0, 99999999)")
    time.sleep(1)

# n: 스크롤할 횟수 설정
n = 10
i = 0
while i < n:
    scroll_down(driver)
    i = i+1
    
# 블로그 글 url들 수집
url_list = []

# URL_raw 크롤링 시작
article_raw = driver.find_elements_by_css_selector(".api_txt_lines.total_tit")
article_raw[0]

# 크롤링한 url 정제 시작
for article in article_raw:
    url = article.get_attribute('href')   
    url_list.append(url)

time.sleep(1)
    
print("url개수: ", len(url_list))

df = pd.DataFrame({'url':url_list})
df

# url_list 저장
df.to_excel("naver_blog_url.xlsx")

url개수:  330


블로그 내용 크롤링

In [4]:
import sys
import os

import pandas as pd
import numpy as np

In [39]:
# "url_list.csv" 불러오기
url_load = pd.read_excel("naver_blog_url.xlsx")

num_list = len(url_load)

print(num_list)
#url_load

330


In [40]:
dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = num_list    # 수집할 글 갯수

# 수집한 url 돌면서 데이터 수집
for i in tqdm_notebook(range(0, number)):
    # 글 띄우기
    url = url_load["url"][i]
    driver = webdriver.Chrome(r"D:\정보현\chromedriver.exe")
    driver.get(url)   # 글 띄우기
    
    # 크롤링
    
    try : 
        # 글의 iframe 접근
        driver.switch_to.frame("mainFrame")

        target_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)

        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)
    
        # 크롤링한 글은 target_info라는 딕셔너리에 담음
        target_info['content'] = content_str
        

        # 각각의 글은 dict라는 딕셔너리에 담음
        dict[i] = target_info
        #time.sleep(1)


        # 글 하나 크롤링 후 크롬 창 닫기
        driver.close()       
    
    # 에러나면 현재 크롬창을 닫고 다음 글(i+1)로 이동
    except:
        driver.close()
        time.sleep(1)
        continue
    
    # 중간 저장 (없어도됨)
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')   # 한글은 오류날 수 있어서 encoding 추가
        time.sleep(3)

print('수집한 글 갯수: ', len(dict))
#print(dict)

C:\Users\minno\AppData\Local\Temp\ipykernel_19696\303234191.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(0, number)):


  0%|          | 0/330 [00:00<?, ?it/s]

수집한 글 갯수:  298


In [41]:
# 판다스화
result_df = pd.DataFrame.from_dict(dict, 'index')
#result_df

# 엑셀 저장
result_df.to_excel("naver_blog_content.xlsx", encoding='utf-8-sig')

액셀 데이터 텍스트 파일로 변환하기

In [42]:
#엑셀 파일을 읽어오는 소스 코드는 
tf=pd.read_excel('naver_blog_content.xlsx', usecols = ['content'])
tf.to_csv("content.txt", index=True)

형태소 분석

In [43]:
#파일 오픈
of = open("content.txt","r",encoding="utf-8")
data=of.read()
data

',content\n0,"#어버이날 #생일선물 #어버이날선물 #결혼기념일선물 #50대엄마생신선물 #부모님결혼기념일선물 #40대엄마생일선물 #칠순선물 #할머니선물 #40대여자선물 #60대엄마생신선물 #엄마생일선물 #엄마선물 #엄마생신선물 #어르신선물 #50대엄마생일선물 #어른선물 #엄마환갑선물 #장모님생신선물 #50대엄마선물 #40대여자생일선물 #50대여자선물 #회갑선물 #남자친구어머니선물 곧 돌아오는 가정의 달에\n어버이날, 스승의날, 장모님 등 감사함을 표시하고자 선물을 할때\n40대 50대 60대 여성 여자분들이 예쁘게 입을 수 있는 옷을 추천하고자 합니다.\n그림 아래에 상품을 구입할 수 있는 주소가 있으니\n상품 정보가 궁금하신 분들은 들어가셔서 확인해보세요~ 후드 나염 배색 자켓\n부모님 생일 또는 어버이날에 선물 하기 좋은 후드 나염 배색 자켓입니다.\n봄 여름 가을 산행이나 나들이 가실때 편안하게 입으실수 있어요 https://smartstore.naver.com/allthatcontent/products/6528841644  플라워 나염 블라우스\n부모님 생일 또는 어버이날에 선물 하기 좋은 플라워 나염 블라우스입니다.\n부모님께서 나들이 가실때 예쁘게 입으실수 있어요 https://smartstore.naver.com/allthatcontent/products/6537170815  프라다 양가죽 콤비 하프코트\n부모님 생일 또는 어버이날에 선물 하기 좋은 프라다 양가죽 콤비 하프코트입니다.\n부모님께서 나들이 가실때 편안하게 입으실수 있어요 https://smartstore.naver.com/allthatcontent/products/6528877012  후드 원피스\n부모님 생일 또는 어버이날에 선물 하기 좋은 후드 원피스입니다.\n부모님께서 평소에 예쁘게 입으실수 있어요 https://smartstore.naver.com/allthatcontent/products/6537277365  진주 장식 하프코트\n부모님 생일 또는 어버이날에 선물 

https://thinkwarelab.wordpress.com/2016/08/30/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%98%95%ED%83%9C%EC%86%8C-%EB%B6%84%EC%84%9D%EC%9C%BC%EB%A1%9C-%EC%9B%8C%EB%93%9C%ED%81%B4%EB%9D%BC%EC%9A%B0%EB%93%9C-%EA%B7%B8%EB%A6%AC%EA%B8%B0/

In [44]:
#!pip install konlpy
from konlpy.tag import Okt
okt=Okt()  #Okt 객체를 생성

nouns = okt.nouns(data)

In [45]:
#명사 추출
nouns

['곧',
 '가정',
 '달',
 '어버이날',
 '스승의날',
 '장모님',
 '등',
 '표시',
 '선물',
 '때',
 '여성',
 '여자',
 '입',
 '수',
 '옷',
 '추천',
 '그림',
 '아래',
 '상품',
 '구입',
 '수',
 '주소',
 '상품',
 '정보',
 '분',
 '확인',
 '후드',
 '나염',
 '배색',
 '자켓',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '후드',
 '나염',
 '배색',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '때',
 '플라워',
 '나염',
 '블라우스',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '플라워',
 '나염',
 '블라우스',
 '부모님',
 '나들이',
 '때',
 '프라다',
 '가죽',
 '콤비',
 '하프',
 '코트',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '프라다',
 '가죽',
 '콤비',
 '하프',
 '코트',
 '부모님',
 '나들이',
 '때',
 '후드',
 '원피스',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '후드',
 '원피스',
 '부모님',
 '평소',
 '진주',
 '장식',
 '하프',
 '코트',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '진주',
 '장식',
 '하프',
 '코트',
 '부모님',
 '나들이',
 '때',
 '타이',
 '분리',
 '셔츠',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '타이',
 '분리',
 '셔츠',
 '부모님',
 '나들이',
 '때',
 '후드',
 '탈',
 '부착',
 '진주',
 '장식',
 '자켓',
 '부모님',
 '생일',
 '어버이날',
 '선물',
 '후드',
 '탈',
 '부착',
 '진주',
 '장식',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '때',
 '벨티드',
 '트렌치코트',
 '부모님',
 '생일',
 

In [76]:
#불용어 처리
stopword = ['선물','어린이날','날','여자','아이','어린이','추천', '엄마', '나', '구매', '때', '수', '것', '더', '이', '저', '가격', '말', 
            '그', '생각', '사용', '제', '제품', '우리', '친구', '거', '조카', '요즘', '등', '위', '요', '집', '하나', '준비', '전', '가지', 
           '곳', '중', '때문', '사진', '할인', '브랜드', '남자', '내', '정말', '번', '모습', '행사', '영화', '사람', '어버이날', '부모님',
            '피부', '박스', '건강', '분', '느낌', '마음', '기능', '안', '고민', '이번', '아빠', '갱년기', '여성', '도움', '해', '성분', 
            '섭취', '관리', '달','선택', '바로', '건', '걸', '포장', '맛', '도', '포', '몸', '디자인', '선생님', '스승의날', '스승', '골드',
           '다니엘', '웰링턴', '향', '착용', '어린이집', '세트', '주문', '개', '생일', '로', '시간', '또', '후', '기분', '캡슐', '카네이션',
            '스트랩', '남자친구', '부담', '오늘', '위해', '감사', '가장', '가정', '리본', '정도', '게', '학생', '학교', '기억', '남성', 
            '수분', '감', '케어', '컬러', '코드', '데', '다이얼', '여자친구', '일리', '가죽', '좀', '남편', '조금', '환', '물', '프로바이오틱스', 
            '나이', '아버지', '저희', '효과', '그냥', '생신', '사실', '부분', '경우', '면도', '장', '에스트로겐', '메노락토', '결혼기념일', 
            '줄', '원료', '꽃집', '꽃다발', '어머니', '문의', '부모', '제작', '졸업', '플라워', '사이즈', '일', '결혼', '아내', '현재', 
            '지금', '시작', '가족', '부부', '김자', '보고', '알', '왜', '살', '이제', '아들']
ko = [i for i in nouns if i not in stopword]
ko

['곧',
 '장모님',
 '표시',
 '입',
 '옷',
 '그림',
 '아래',
 '상품',
 '구입',
 '주소',
 '상품',
 '정보',
 '확인',
 '후드',
 '나염',
 '배색',
 '자켓',
 '후드',
 '나염',
 '배색',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '나염',
 '블라우스',
 '나염',
 '블라우스',
 '나들이',
 '프라다',
 '콤비',
 '하프',
 '코트',
 '프라다',
 '콤비',
 '하프',
 '코트',
 '나들이',
 '후드',
 '원피스',
 '후드',
 '원피스',
 '평소',
 '진주',
 '장식',
 '하프',
 '코트',
 '진주',
 '장식',
 '하프',
 '코트',
 '나들이',
 '타이',
 '분리',
 '셔츠',
 '타이',
 '분리',
 '셔츠',
 '나들이',
 '후드',
 '탈',
 '부착',
 '진주',
 '장식',
 '자켓',
 '후드',
 '탈',
 '부착',
 '진주',
 '장식',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '벨티드',
 '트렌치코트',
 '벨티드',
 '트렌치코트',
 '나들이',
 '코디',
 '지퍼',
 '프라다',
 '자켓',
 '지퍼',
 '프라다',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '콤비',
 '자켓',
 '콤비',
 '자켓',
 '봄',
 '여름',
 '가을',
 '산행',
 '나들이',
 '르',
 '니트',
 '배색',
 '프라다',
 '하프',
 '후드',
 '코트',
 '르',
 '니트',
 '배색',
 '프라다',
 '하프',
 '후드',
 '코트',
 '나들이',
 '코디',
 '장모님',
 '추후',
 '계속',
 '얼마',
 '제대로',
 '환갑',
 '산이',
 '밭',
 '시골',
 '땀',
 '눈',
 '얼굴',
 '뭔가',
 '봄',
 '만',
 '스킨',
 '턱',
 '입',
 '주변',
 '올인원',
 '화장

In [78]:
#빈도수 체크
from collections import Counter
count = Counter(ko)
#count

#상위 3 출력
count.most_common(3)

[('꽃', 659), ('돈', 194), ('케이크', 112)]

https://dalulu.tistory.com/108
http://bigdata.dongguk.ac.kr/lectures/DB/_book/python%EC%97%90%EC%84%9C-mysql%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%9D%98-%EC%A0%91%EA%B7%BC.html

In [79]:
#딕셔너리 생성 후 키값과 value값 분리
tag_count=[]
tags=[]

for n,c in count.most_common(3):
    dics={'tag': n, 'count':c}
    tag_count.append(dics)
    tags.append(dics['tag'])

https://hyun-am-coding.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-DB%EC%97%90-%EC%A0%80%EC%9E%A5%ED%95%98%EA%B8%B0
https://codechacha.com/ko/python-mysql-select-order/

In [77]:
!pip install mysqlclient
!pip install pymysql

In [271]:
#DB 셍성, 테이블 생성, 데이터 추가(처음 코드를 실행할 때 사용)

import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
    # charset="utf-8"
)
# 커서 생성하기
cursor = conn.cursor()

# 실행할 때마다 다른값이 나오지 않게 테이블을 제거해두기
cursor.execute("DROP TABLE IF EXISTS recommend")

# 테이블 생성하기
cursor.execute("CREATE TABLE recommend (`rank` int, situation text, gender text, age text, present text)")

# 데이터 저장하기
situation = "Children"    #Children, Parents, Teacher, Birthday, Wedding
gender = "woman"    #man, woman
age = "ten"    #ten, twenty, forty, sixty 
i=1

for tag in tag_count:
    cursor.execute(f"INSERT INTO recommend VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1

# 커밋하기
conn.commit()

# 연결종료하기
conn.close()

https://codechacha.com/ko/python-mysql-insert-update-delete/

In [80]:
#데이터 추가(DB 및 테이블 생성 후 데이터만 추가하기)

import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 커서 생성하기
cursor = conn.cursor()

sql = "INSERT INTO recommend (rank, situation, gender, age, present) VALUES (%d, %s, %s, %s, %s)"

# 데이터 저장하기
situation = "Wedding"
gender = "man"
age = "sixty"
i=1

for tag in tag_count:
    cursor.execute(f"INSERT INTO recommend VALUES({i},\"{situation}\", \"{gender}\", \"{age}\", \"{tag['tag']}\")")
    i+=1


# 커밋하기
conn.commit()

In [81]:
import MySQLdb

conn = MySQLdb.connect(
    user="crawl_usr",
    passwd="Test001",
    host="localhost",
    db="crawl_data"
)

# 테이블 불러옴
with conn:
    with conn.cursor() as cursor:
        #cursor.execute("SELECT * FROM recommend")
        cursor.execute("SELECT * FROM recommend WHERE situation LIKE %s AND age LIKE %s", ['Teacher', 'sixty'])
        result=cursor.fetchall()
        print(result)


((1, 'Teacher', 'woman', 'sixty', '꽃'), (2, 'Teacher', 'woman', 'sixty', '콜라겐'), (3, 'Teacher', 'woman', 'sixty', '크림'), (1, 'Teacher', 'man', 'sixty', '꽃'), (2, 'Teacher', 'man', 'sixty', '과일'), (3, 'Teacher', 'man', 'sixty', '편지'))
